## Coverage
1. Writing a file from local file system to hadoop
2. Starting Spark session
3. Creating RDDs 
  - RDD data types
  - Reading from local file system or hdfs (not working in jupyter)
  - From other RDDs using transformations
    - Using python functional programming style to create RDD transformations (map, filter, reduce) 
    - map, reduce, filter in python
4. Checking Lineage (DebugtoString)  
5. Memory Management
6. Additional transformations on RDDs based on types
  - paired RDDs
  - Other useful transformations
7. Paired RDD Operations and case 
8. Memory Management : Persistence, Caching, Serialization  
9. Optimization in Spark

#### 1. Writing a file from local file system to hadoop

In [ ]:
!pwd
! touch purple_cow.txt
!ls
with open('purple_cow.txt', 'w') as con:
    con.write("""I've never seen a purple cow
    I never hope to see one;
    But, i can tell you, anyhow,
    I'd rather see than be one""")

!cat purple_cow.txt
#! hadoop fs -ls fractalUS
! hadoop fs -put purple_cow.txt fractalUS/
!hadoop fs -cat fractalUS/purple_cow.txt

#### 2. Starting Spark session and accessing spark context, which is used to create and access RDDs

spark 2.3.1 use Spark Session for entry point instead of spark context
Spark Session can be use use to create sql context (for Data Frames) as well as hivcontext, spark context
for RDDs

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rdds').getOrCreate()
sc = spark.sparkContext

#from pyspark import SparkContext
#sc = SparkContext(appName= 'rdds')
#rdd = sc.textFile('/home/sumad/purple_cow.txt')
#rdd.take(2)

spark.version

#### 3. Creating RDDs
- Types of RDDs
- from files, collections etc 
- Create RDDs using common transformations - map, filter, reduce 
  - Use of functional programming : 
    - pass function to a function, and use of anonymous functions
    - rdd methods support functional programming, i.e they take functions as input and apply them 
    over each line in data
- See the DAG of transformations 
- DAGs:
   - provide lazy evaluations 
   - start computation from point of failure 
   - all transformations created RDDs in memory (unlike write to disk in map reduce)
- Row wise transformations where possible   
  - Where possible transformations happen row by row, i.e a row is taken through all transformations to an 
  action, instead of all blocks being acted by a single transformation at once

#### 3.1 Types of RDD 
Can hold any type of element
- Primitive type( integerm character etc)
- Sequence RDDs ( from dics, lists, tuples)
- Mixed data dype
- Pair RDDs ( support special transformations)
- Double RDDs (support numeric transformations)

#### 3.2 Creating RDDs

In [ ]:
# from python collections
sc.parallelize([1,2,3,4])

# From text files 
## textFile only works with line delimited files
## Each line in text file is a new record in RDD
sc.textFile('dir/*.log') # all files in the dir with .log

# Xml or json files
## As there is no new line delimiter
sc.wholeTextFiles(dir) # reads whol file as RDD with whole file as a single element, size should be checked

Other input and Output formats available 
https://spark.apache.org/docs/2.3.1/api/python/pyspark.html#pyspark.RDD

#### 3.3 From transformations

#### map, filter, reduce in python 
- find sum of squares of first 10 integers that are even
- functional programming : functions serve as i/p and o/p units, and are chained

In [ ]:
from functools import reduce

l = range(1,11,1)
f = filter(lambda x: x%2 ==0, l)
m = map(lambda x : x**2, f)
r = reduce(lambda x,y: x+ y, m)
print(r)

In [ ]:
#### Convert all words in the text file into caps, and count lines that start with I, then do a word count

rdd = sc.textFile('/user/sumad/fractalUS/purple_cow.txt')
# act on each line of text file, each line is a string 
# filter needs a function to operate on each line and return boolean
rdd2 = rdd.map(lambda x: x.upper()) \
   .filter(lambda x: x.strip().startswith('I')) 
rdd2.count()
#3

In [ ]:
# Using print improves readability of DAG
## show starting from text file two RDDs are created
print(rdd2.toDebugString())
"""
(2) PythonRDD[4] at RDD at PythonRDD.scala:49 []
 |  /user/sumad/fractalUS/purple_cow.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /user/sumad/fractalUS/purple_cow.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []
 """

In [ ]:
 #Word Count
rdd3 = rdd.map(lambda x: x.strip()).flatMap(lambda x:x.split(' ')).map(lambda x: x.upper())\
.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
rdd3.collect()

#### 4. Checking lineage 
- Can see shuffle operation because of reduceByKey

In [ ]:
"""
2) PythonRDD[16] at collect at <stdin>:1 []
 |  MapPartitionsRDD[15] at mapPartitions at PythonRDD.scala:129 []
 |  ShuffledRDD[14] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[13] at reduceByKey at <stdin>:2 []
    |  PythonRDD[12] at reduceByKey at <stdin>:2 []
    |  /user/sumad/fractalUS/purple_cow.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
    |  /user/sumad/fractalUS/purple_cow.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []
""";

#### 5. Memory Management 
- When RDD's are created, they are created in RAM. What happens if memory is used up. 
  - RDDs are ejected by LRU (Least recently used) 
  - Persisting RDDs affects this rule, to be covered later in detail. 
  - **Also, if RDDs are not persisted, then they are cleaned from the memory after an action is called**

#### 6. Additional transformations (key ones)
- List of all
  - https://spark.apache.org/docs/2.3.1/api/python/pyspark.html#pyspark.RDD

- Operate and return single RDD
  - flatMap : first breaks sequence inside each RDD element, then, combines all elements in a single sequence,so
    retutns a single RDD with single element.
  - map , filter, reduce
  - foreach, distinct , top(n), first
  - min, max, mean, stddev
  - sample, randomSplit, 
- Operate on two RDDs
  - zip, intersection, union , subtract 
  - join 
- Operate on Paired RDD 
  - countByKey
  - groupByKey, reduceByKey,aggregateByKey : latter two perform better than first for grouping and aggregating
- Return paired RDD 
  - countByValue : paired RDD with unique element and its count
- Explicit repartitioning or operations by partition
  - mapPartitions :
  - glom : combines elements in each partition into a unique list of elements
  - colesce : returns rdd by repartioning to specified no. of partitions 
  - foreachPartition 
  - getNumPartitions
  - partitionBy, repartition (coalesce can prevent a shuffle operation)

#### 7. Paired RDD operations and case study
- pair RDDs have k, v pairs 
- many methods available : grouping, counting, aggregating over groups , join etc
- Common flow of operations. 
  - Creating pair RDDs
    - map, flatMap

#### Case Study, using telecom weblogs
1. What are the potential marketable customers for MMS product
  - jpg, mp3, mp4 type of message
  - Data given in log file:  ip address, phone number,  time, file type, address, file link,


In [ ]:
ip_rdd = sc.textFile('/user/sumad/weblogs/*.log')
#ip_rdd.take(2)

In [ ]:
[u'3.94.78.5 - 69827 [15/Sep/2013:23:58:36 +0100] "GET /KBDOC-00033.html HTTP/1.0" 200 14417 
 "http://www.loudacre.com"  "Loudacre Mobile Browser iFruit 1"', 
 u'3.94.78.5 - 69827 [15/Sep/2013:23:58:36 +0100] "GET /theme.css HTTP/1.0" 200 3576 
 "http://www.loudacre.com"  "Loudacre Mobile Browser iFruit 1"']

In [ ]:
# 1. By customer id, day, can we get count of mms type messages sent
# 2. Get by customer id, average mms messages per day, get top 10 

extract_id_day_mms = lambda x: (x[2], x[3][1:12])
def mms(x):
    if(('html' in x) | ('png' in x) | ('css' in x)):
        return(1)
    else:
        return(0)    
ip_rdd2 = ip_rdd.filter(mms).map(lambda x: x.split(' ')).map(extract_id_day_mms)
by_cust = ip_rdd2.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y:x+y)
#by_cust.take(5)
by_cust_dt = ip_rdd2.map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y)\
.map(lambda x:(x[0][0],x[1])).reduceByKey(lambda x,y:x+y)
#by_cust_dt.take(5)
avg_per_day = by_cust.leftOuterJoin(by_cust_dt)\
.map(lambda x: (x[0],x[1][0]/x[1][1]))\
.sortBy(keyfunc = lambda x:x[1],ascending = False)
avg_per_day.take(10)

In [ ]:
print(avg_per_day.toDebugSchema())

#### 8. Memory Management - Persistence, Caching, Serialization
- When RDD's are created, they are created in RAM. What happens if memory is used up. 
  - RDDs are ejected by LRU (Least recently used) 
  - If RDD is persisted, then it is given priority 
- Persist vs Cache 
  - Notes
  - 6 options with consideration for : Memory/Disk , Serialization or not, Copying partitions or not. 
    - allow fo considering available memory vs disk, conserve memory ( by serializing) but increasing 
    processing time for deserializing , replication to revover from failure if persisted rdd is lost (saves time,
    when a long pipeline is built)
  - Link to documentation : https://spark.apache.org/docs/2.3.0/rdd-programming-guide.html#rdd-persistence

In [ ]:
from pyspark import StorageLevel
test = sc.parallelize([1,2,3,4,5,6])
test1 = test.map(lambda x : x**2)
test2 = test1.filter(lambda x : x%2 ==0)
test2.collect()

test1.persist(storageLevel = StorageLevel.MEMORY_ONLY)
test2.collect()
test2.collect() # this is faster, because persistence take effect first time after 
                # rdd is specified for persistence

#### 9. Optimization in Spark
- Stages and Tasks 
   - Stages created whenever there is partitioning 
   - Optimization is focussed on reducing stages as much as possible, involves design discussions from architects 
   - Short circuiting

In [ ]:
# Check, the below operation should not cause a repartition , and hence should be a stage
extract_id_day_mms = lambda x: (x[2], (x[3][1:12],1))
def mms(x):
    if(('html' in x) | ('png' in x) | ('css' in x)):
        return(1)
    else:
        return(0)
    
ip_rdd = sc.textFile('/user/sumad/weblogs/*.log')
ip_rdd2 = ip_rdd.filter(mms).map(lambda x: x.split(' ')).map(extract_id_day_mms)
ip_rdd2.take(5)
 print(ip_rdd2.toDebugString())
    
"""
(182) PythonRDD[3] at RDD at PythonRDD.scala:49 []
  |   /user/sumad/weblogs/*.log MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
  |   /user/sumad/weblogs/*.log HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []
  """

#### Stages and tasks with times can be better seen on Spark UI 
- Giving spark the full job , lets it optimize the total run time  
- Big executions, should be given as spark submit presumably, rather than interactively, ti optimize time

In [ ]:
"""
print(avg_per_day.toDebugString())

(182) PythonRDD[28] at RDD at PythonRDD.scala:49 []
  |   MapPartitionsRDD[22] at mapPartitions at PythonRDD.scala:129 []
  |   ShuffledRDD[21] at partitionBy at NativeMethodAccessorImpl.java:0 []
  +-(182) PairwiseRDD[20] at sortBy at <stdin>:3 []
      |   PythonRDD[19] at sortBy at <stdin>:3 []
      |   PartitionerAwareUnionRDD[16] at union at NativeMethodAccessorImpl.java:0 []
      |   PythonRDD[14] at RDD at PythonRDD.scala:49 []
      |   MapPartitionsRDD[5] at mapPartitions at PythonRDD.scala:129 []
      |   ShuffledRDD[4] at partitionBy at NativeMethodAccessorImpl.java:0 []
      +-(182) PairwiseRDD[3] at reduceByKey at <stdin>:1 []
          |   PythonRDD[2] at reduceByKey at <stdin>:1 []
          |   /user/sumad/weblogs/*.log MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
          |   /user/sumad/weblogs/*.log HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []
      |   PythonRDD[15] at RDD at PythonRDD.scala:49 []
      |   MapPartitionsRDD[13] at mapPartitions at PythonRDD.scala:129 []
      |   ShuffledRDD[12] at partitionBy at NativeMethodAccessorImpl.java:0 []
      +-(182) PairwiseRDD[11] at reduceByKey at <stdin>:2 []
          |   PythonRDD[10] at reduceByKey at <stdin>:2 []
          |   MapPartitionsRDD[9] at mapPartitions at PythonRDD.scala:129 []
          |   ShuffledRDD[8] at partitionBy at NativeMethodAccessorImpl.java:0 []
          +-(182) PairwiseRDD[7] at reduceByKey at <stdin>:1 []
              |   PythonRDD[6] at reduceByKey at <stdin>:1 []
              |   /user/sumad/weblogs/*.log MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
              |   /user/sumad/weblogs/*.log HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []
              """